First, we'll download a 1/3-arcsecond DEM of the region we're interested in.
The DEM comes from the Oregon Lidar Consortium, which is part of the Oregon Department of Geology and Mineral Industries (DOGAMI).
The DOGAMI website has an interactive online [viewer](https://gis.dogami.oregon.gov/maps/lidarviewer/) and download utility for LiDAR data.
The code below uses a library called [pooch](https://www.fatiando.org/pooch) to describe what file we want to get and from where.

In [ ]:
import pooch

url = "https://www.oregongeology.org/pubs/ldq/"
archive_filename = "LDQ-43124D1.zip"
checksum = "cb1fcb26fbb6e84640a554fb2287c619cfe6f54bc81a6423624273ceb21f7647"
dem = pooch.create(
    path=pooch.os_cache("hillslope"),
    base_url=url,
    registry={archive_filename: checksum},
)

Next we'll actually fetch the raw data, unzip it, and extract a `.adf` file (an ArcInfo binary format) containing the actual DEM.

In [ ]:
import requests

try:
    downloader = pooch.HTTPDownloader(progressbar=True)
    files = dem.fetch(
        archive_filename,
        processor=pooch.Unzip(),
        downloader=downloader,
    )
except requests.exceptions.SSLError:
    downloader = pooch.HTTPDownloader(progressbar=True, verify=False)
    files = dem.fetch(
        archive_filename,
        processor=pooch.Unzip(),
        downloader=downloader,
    )

In [ ]:
filename = [
    f for f in files if "South Coast" in f and "Bare_Earth" in f and "w001001.adf" in f
][0]

print(filename)

Now we can open the DEM using the library [rasterio](https://rasterio.readthedocs.io/en/latest/).

In [ ]:
import rasterio

dataset = rasterio.open(filename, "r")

We can query this dataset object for the coordinate reference system (CRS) and a bounding box.

In [ ]:
print(dataset.crs)
print(dataset.bounds)

The data aren't reported in a simple obvious CRS like a latitude-longitude, but rather a specialized CRS with code EPSG:3644.
**Note that the data use measurements in feet, not in meters.**
I used [pyproj](https://pyproj4.github.io/pyproj) to roughly figure out a bounding box from the lat-lon coordinates given in Roering's 2008 paper of 43.464${}^\circ$N, 124.119${}^\circ$W and by eyeballing roughly the limits of the domain from figure 5 in the paper.
Rather than read in the whole DEM tile, which contains much more area than we care about, we'll do a windowed read of just this small patch.

In [ ]:
bbox = {
    "left": 349750.0,
    "right": 353350.0,
    "bottom": 644360.0,
    "top": 647550.0,
}

window = rasterio.windows.from_bounds(
    **bbox, transform=dataset.transform, width=dataset.width, height=dataset.height
)

elevation = dataset.read(indexes=1, window=window)[::-1, :]
transform = dataset.window_transform(window)

To make sure we did everything correctly, we'll make a 3D plot of the elevation.
The slope we're interested in runs down the middle of the figure below.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(projection="3d")
ax.view_init(15, 100)
x = np.linspace(bbox["left"], bbox["right"], elevation.shape[1])
y = np.linspace(bbox["bottom"], bbox["top"], elevation.shape[0])
ax.set_box_aspect((np.ptp(x), np.ptp(y), np.ptp(elevation)))
X, Y = np.meshgrid(x, y)
ax.plot_surface(X, Y, elevation, linewidth=0, antialiased=False)

Next we'll use the [pysheds](https://mattbartos.com/pysheds/) package to calculate the catchment area throughout the domain.
The original results that we aim to reproduce here exclude any parts of the domain where the catchment area exceeds 250 m${}^2$, i.e. to capture the hillslopes and exclude valleys where fluvial transport dominates.

In [ ]:
import pysheds, pysheds.grid
import pyproj

crs = pyproj.Proj("epsg:3644")
grid = pysheds.grid.Grid(affine=transform, shape=elevation.shape, crs=crs)

The steps here are to (1) add the elevation data, (2) fill depressions that won't drain out of the domain, (3) remove flat parts of the DEM where a flow direction can't meaningfully be defined, and finally (4) calculate flow directions using the D${}^\infty$ routing algorithm from [Tarboton 1997](https://doi.org/10.1029/96WR03137).

In [ ]:
grid.add_gridded_data(elevation, "elevation", affine=transform, crs=crs)
grid.fill_depressions(data="elevation", out_name="flooded_elevation")
grid.resolve_flats(data="flooded_elevation", out_name="inflated_elevation")
grid.flowdir(data="inflated_elevation", out_name="flow_dir", routing="dinf")

Now that we've calculated the flow directions, we can calculate the accumulation or catchment areas.

In [ ]:
areas = grid.cell_area(as_crs=crs, inplace=False)
weights = (areas / areas.max()).ravel()
grid.accumulation(
    data="flow_dir", routing="dinf", weights=weights, out_name="accumulation"
)

The plot below shows the catchment area.
The dark blue areas are rivers and valleys, the bright yellow are ridge tops.

In [ ]:
from matplotlib.colors import LogNorm

fig, axes = plt.subplots(figsize=(8, 8))
norm = LogNorm(vmin=1, vmax=grid.accumulation.max() + 1)
image = axes.imshow(grid.accumulation + 1, extent=extent, cmap="viridis_r", norm=norm)
fig.colorbar(image)

This plot shows the elevation with the rivers and valleys masked out.

In [ ]:
import numpy.ma as ma

maxval = 300.0
mask = grid.accumulation > maxval
z = ma.masked_array(grid.elevation, mask=mask)
norm = LogNorm(vmin=1, vmax=maxval + 1)
fig, axes = plt.subplots(figsize=(8, 8))
image = axes.imshow(z + 1, extent=extent)
fig.colorbar(image)